In [1]:
import tensorflow as tf
import numpy as np
import math
na = np.newaxis
import matplotlib.pyplot as plt

# import helper functions from utils.py:
from utils import tf_tril_indices, load_lookup_table, table_lookup_op_parallel, get_scp_samples, build_graph

# Planning:

- refactor code / make it more clear
- test on simple example and compare real / optimized distributions (define real dist-> sample data -> calc approx -> compare)
- test on real dataset

### TODOS:
- Add Tensorboard logging and remove all the prints
- Remove hardcoded num_inducing points (problem: using tf.shape(Z_ph)[0] returns a tensor, variables cannot be initialized with unknown shape)
- Define hyperparameters as variables and add two optimizers with var_lists to optimize with/without hyperparameters 

# Main Graph

In [2]:
## ######## ##
# PARAMETERS #
## ######## ##
log_dir = 'logs'
max_iterations = 3000
# inducing point location
Zx = np.linspace(0, 20, 11)[:,na]
Zy = np.linspace(0, 10, 11)[:,na]

Z = np.concatenate((Zx,Zy),axis=1)

X = np.random.rand(10,2)*10

wr_means = []
wr_covar = []


## ######### ##
# BUILD GRAPH #
## ######### ##
tf.reset_default_graph()
lower_bound, merged, Z_ph, u_ph, X_ph, m, S = build_graph(Z.shape[0])

with tf.name_scope('optimization'):
    train_step = tf.train.GradientDescentOptimizer(0.0001).minimize(-lower_bound)

## ########## ##
# OPTIMIZATION #
## ########## ##
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(log_dir, sess.graph)
    
    for i in range(max_iterations):
        _, m_val, S_val, summary = sess.run([train_step, m, S, merged], feed_dict={Z_ph:Z, u_ph:0.,X_ph:X})
        writer.add_summary(summary, i)

plt.show()

In [3]:
# numpy ard_kernel calculation
def ard_kernel_np(X1, X2, gamma = 1., alphas = None):
    
    if alphas == None:
        alphas = np.ones(X1.shape[1])
    
    return gamma * np.prod(np.exp( - (X1[:,None,:] - X2[None,:,:])**2 / (2 * alphas[None,None,:])), axis=2)

In [4]:
# TEST

r_mean = np.zeros(num_inducing_points)
r_cov = ard_kernel_np(Z, Z)

print(np.allclose(r_mean, m_val))
print(np.allclose(r_cov, S_val))

print(np.sum(r_cov - S_val)**2)

# values not allclose but almost the same

NameError: name 'num_inducing_points' is not defined

### example sampling:

In [ ]:
with tf.name_scope('prior_sampling'):
    cov  = K_zz
    mean = u_ph * tf.ones([num_inducing_points])
    ind_point_dist = tf.contrib.distributions.MultivariateNormalFullCovariance(mean, cov)
    samples = ind_point_dist.sample(n_ph)
    

n_ph = tf.placeholder(tf.int32,   [],           name='number_samples')


# sampling
num_samples = 10

with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    writer = tf.summary.FileWriter('logs', sess.graph)
    sample_res, mean_res, cov_res = sess.run([samples, mean, cov], feed_dict={Z_ph:Z, u_ph:0., n_ph:num_samples})

In [ ]:
for i in range(num_samples):
    plt.plot(Z, sample_res[i])
plt.show()

## 1-D example: 